### Noise Code: Measurements from Keithley Multimeter
1. Goal of program is to just take voltage readings from Keithley multimeter via GPIB interface with some time delay and write out to a text file (for now). 
2. Things to keep in mind: need to install both pyvisa and pyvisa-py (backend) and specify pyvisa-py when using the Resource manager; also if module not found, change python interpreter to correct one (ie, like Conda)

In [1]:
import pyvisa
import numpy as np
import pandas as pd

#### Find out the GPIB, Declare Resource Manager, etc

In [2]:
# declare resource manager object called rm and list resources to find multimeter
rm = pyvisa.ResourceManager('@py')

In [16]:
# Create instrument variable multimeter 
multimeter = rm.open_resource('GPIB0::16::INSTR')

ValueError: Please install linux-gpib (Linux) or gpib-ctypes (Windows, Linux) to use this resource type. Note that installing gpib-ctypes will give you access to a broader range of funcionality.
No module named 'gpib'

#### Initialize Machine For Measurements

In [ ]:
# experimental parameters
time_interval_ms = 500
number_readings = 50

# SCPI (standard commands for programmable instruments)

multimeter.write("status:measurement:enable 512; *sre 1")
multimeter.write("sample:count %d" % number_of_readings)
multimeter.write("trigger:source bus")
multimeter.write("trigger:delay %f" % (time_interval_ms / 1000.0))
multimeter.write("trace:points %d" % number_readings)
multimeter.write("trace:feed sense1; trace:feed:control next")

#### Perform the Measurements

In [ ]:
multimeter.write("initiate")
multimeter.assert_trigger()
multimeter.wait_for_srq()

#### Get array of voltage values

In [ ]:
voltages = keithley.query_ascii_values("trace:data?", container=numpy.array)

#### Save voltage values out to a csv file

In [33]:
np.savetxt('File_Name.csv', voltages, fmt='%.7e', delimiter=',')

NameError: name 'voltages' is not defined

#### Some useful things maybe

In [24]:
def mean(data_array):
    result = 0
    for i in range(len(data_array)):
        result = data_array[i] + result 
    return result/len(data_array)

def variance(data_array):
    # returns population variance
    return mean((data_array - mean(data_array))**2)
    
def uncertainty(data_array):
    # returns uncertainty defined as max value - min value divided by 2
    return (np.max(data_array) - np.min(data_array))/2
    
def uncertainty_mean(data_array):
    # returns uncertainty in the mean: max value - min value, divided by 2 sqrt(N)
    return (np.max(data_array) - np.min(data_array))/2/np.sqrt(len(data_array))

### Data Analysis Section